## Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

### Шаг 1. Откройте файл с данными и изучите общую информацию. 

In [1]:
import pandas as pd

bank_data = pd.read_csv('/datasets/data.csv')

bank_data.head(5)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


In [2]:
#общая информация
bank_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [3]:
#считаем количество строк - дубликатов
print('Число дубликатов в таблице:', bank_data.duplicated().sum())

Число дубликатов в таблице: 54


In [4]:
#столбцы с некорректными данными
bank_data['education'].unique()

array(['высшее', 'среднее', 'Среднее', 'СРЕДНЕЕ', 'ВЫСШЕЕ',
       'неоконченное высшее', 'начальное', 'Высшее',
       'НЕОКОНЧЕННОЕ ВЫСШЕЕ', 'Неоконченное высшее', 'НАЧАЛЬНОЕ',
       'Начальное', 'Ученая степень', 'УЧЕНАЯ СТЕПЕНЬ', 'ученая степень'],
      dtype=object)

In [5]:
bank_data['dob_years'].unique()

array([42, 36, 33, 32, 53, 27, 43, 50, 35, 41, 40, 65, 54, 56, 26, 48, 24,
       21, 57, 67, 28, 63, 62, 47, 34, 68, 25, 31, 30, 20, 49, 37, 45, 61,
       64, 44, 52, 46, 23, 38, 39, 51,  0, 59, 29, 60, 55, 58, 71, 22, 73,
       66, 69, 19, 72, 70, 74, 75])

### Вывод

1) В файле находятся данные о 21525 клиентах, каждый из которых описан 12 параметрами: 
         children,  days_employed, dob_years, total_income - количественные,остальные 8 - категориальные. 

2) В столбцах days_employed и total_income пристутствуют пропуски у 2174 клиентов - это 10% от общего числа клиентов. Пропуски необходимо заполнить

3) В столбце education данные неоднородные - указаны то прописными буквами, то строчными. Необходимо все данные привести к нижнему регистру

4) В столбце dob_years присутствуют нулевые значения,которые необходимо будет устранить. 

5) В таблице присутствуют дубликаты

6)В столбце days_employed данные указаны в непонятном формате: непонятно, в минутах, в секундах ли, вероятно, это ошибка заполнения или обработки данных. К счастью, этот столбец не понадобится нам исседовании надежности заемщиков, но пропуски в следующем шаге заполним. 


### Шаг 2. Предобработка данных

### Обработка пропусков

In [6]:
#заменим пропущенные значения в total_income на медианные по каждому типу занятости
#определим, в каких типах занятости встречаются пропущенные значения 

income_type = bank_data.loc[bank_data['total_income'].isnull(), 'income_type'].value_counts()
income_type

сотрудник          1105
компаньон           508
пенсионер           413
госслужащий         147
предприниматель       1
Name: income_type, dtype: int64

In [7]:
#найдем медианный уровень дохода для каждого типа занятости
total_income_median = bank_data.groupby('income_type')['total_income'].median().reset_index()
total_income_median

,income_type,total_income
0,безработный,131339.751676
1,в декрете,53829.130729
2,госслужащий,150447.935283
3,компаньон,172357.950966
4,пенсионер,118514.486412
5,предприниматель,499163.144947
6,сотрудник,142594.396847
7,студент,98201.625314


In [8]:
# заменим пропущенные значения в основной таблице
for index, row in total_income_median.iterrows():
    bank_data.loc[(bank_data['total_income'].isnull()) & 
         (bank_data['income_type'] == row[0]), 'total_income'] = row[1]

In [9]:
#заменим нулевые значения в dob_years на медианные по каждому типу занятости
#определим, в каких типах занятости встречаются нулевые значения:
age = bank_data.loc[bank_data['dob_years']==0, 'income_type'].value_counts()
age

сотрудник      55
пенсионер      20
компаньон      20
госслужащий     6
Name: income_type, dtype: int64

In [10]:
#найдем медианный возраст для каждого типа занятости
age_median = bank_data.groupby('income_type')['dob_years'].median().reset_index()
age_median

,income_type,dob_years
0,безработный,38.0
1,в декрете,39.0
2,госслужащий,40.0
3,компаньон,39.0
4,пенсионер,60.0
5,предприниматель,42.5
6,сотрудник,39.0
7,студент,22.0


In [11]:
# заменим нулевой возраст на медианный я в основной таблице
for index, row in age_median.iterrows():
    bank_data.loc[(bank_data['dob_years']==0) & 
         (bank_data['income_type'] == row[0]), 'dob_years'] = row[1]

In [12]:
#заменим пропущенные значения в days_employed на медианные по возрасту
#для этого напишем функцию, которая присвоит каждому клиенту его возрастную категорию
def age_category(age):
    if 18 <= age < 25:
        return '18-24'
    elif 25 <= age < 35:
        return '25-34'
    elif 35 <= age < 45:
        return '35-44'
    elif 45 <= age < 55:
        return '45-54'
    elif 55 <= age < 65:
        return '55-64'
    elif age >= 65:
        return '65+'
    return 'несовершеннолетний'

In [13]:
#применим функцию к столбцу dob_years
bank_data['age_category'] = bank_data['dob_years'].apply(age_category)

In [14]:
#проверка
bank_data['age_category'].value_counts()

35-44    5834
25-34    5100
45-54    4882
55-64    3933
65+       899
18-24     877
Name: age_category, dtype: int64

In [15]:
#найдем медианное значение трудового стажа для каждой возрастной группы 
days_employed = bank_data.groupby('age_category')['days_employed'].median().reset_index()
days_employed

,age_category,days_employed
0,18-24,-744.016267
1,25-34,-1280.124639
2,35-44,-1787.680538
3,45-54,-1768.890854
4,55-64,341062.171460
5,65+,360304.232308


In [16]:
# заменим пропущенные значения в основной таблице
for index, row in days_employed.iterrows():
    bank_data.loc[(bank_data['days_employed'].isnull()) & 
         (bank_data['age_category'] == row[0]), 'days_employed'] = row[1]

In [17]:
#проверяем, что нет пропусков
bank_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 13 columns):
children            21525 non-null int64
days_employed       21525 non-null float64
dob_years           21525 non-null float64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null float64
purpose             21525 non-null object
age_category        21525 non-null object
dtypes: float64(3), int64(4), object(6)
memory usage: 2.1+ MB


### Вывод

Теперь заполнены все строки: 

1) пропущенные значения в уровне дохода клиентов банка заменены на медианные по типу занятости

2) нулевые значения в возрасте заменены на медианные по типу занятости

3) пропущенные значения тудового стажа заменены на медианные по возрастным группам

### Замена типа данных

In [18]:
#просмотрим на типы данных в таблице
bank_data.dtypes

children              int64
days_employed       float64
dob_years           float64
education            object
education_id          int64
family_status        object
family_status_id      int64
gender               object
income_type          object
debt                  int64
total_income        float64
purpose              object
age_category         object
dtype: object

In [19]:
#возраст клиентов и стаж работы записаны вещественными числами - заменим их на целочисленные:
bank_data['days_employed'] = bank_data['days_employed'].astype(int)
bank_data['dob_years'] = bank_data['dob_years'].astype(int)

In [20]:
#проверим, как изменились данные
bank_data.dtypes

children              int64
days_employed         int64
dob_years             int64
education            object
education_id          int64
family_status        object
family_status_id      int64
gender               object
income_type          object
debt                  int64
total_income        float64
purpose              object
age_category         object
dtype: object

### Вывод

Мы заменили данные в колонках days_employed и dob_years были записаны вещественными числами - мы заменили их на целочисленные методом astype (метод to_numeric тут не подходит, потому что мы уже заменили все пропущенные значения)

### Обработка дубликатов

In [21]:
#перед поиском дубликатов приведем все значения в колонке education к нижнему регистру, иначе такие значения как 'среднее' и 'Среднее'
#при поиске дубликатов будут определены как разные значения. 
bank_data['education'] = bank_data['education'].str.lower()

In [22]:
#проверяем результат
bank_data['education'].value_counts()

среднее                15233
высшее                  5260
неоконченное высшее      744
начальное                282
ученая степень             6
Name: education, dtype: int64

In [23]:
#посчитаем количество дубликатов в таблице: 
print('Число дубликатов в таблице:', bank_data.duplicated().sum())

Число дубликатов в таблице: 71


In [24]:
# удалим дупликаты и проверим, что их больше нет:
bank_data = bank_data.drop_duplicates().reset_index(drop = True)

print('Число дубликатов в массиве данных:', bank_data.duplicated().sum())

Число дубликатов в массиве данных: 0


### Вывод

При обработке данных была найдена 71 дублирующаяся строка. Возможно, проблема возникла из-за того, что данные заполняются вручную, что приводит к значениям вроде "среднее" и "Среднее", либо здесь какая-то техническая проблема. Для удаления дубликатов использовали метод drop_duplicates(), который используется для удаления полностью идентичных строк.   

### Лемматизация

In [25]:
#импортируем необходимые библиотеки
from pymystem3 import Mystem
from collections import Counter
m = Mystem()

In [26]:
#выделим текущие варианты целей кредита: 
bank_data['purpose'].value_counts()

свадьба                                   791
на проведение свадьбы                     768
сыграть свадьбу                           765
операции с недвижимостью                  675
покупка коммерческой недвижимости         661
операции с жильем                         652
покупка жилья для сдачи                   651
операции с коммерческой недвижимостью     650
жилье                                     646
покупка жилья                             646
покупка жилья для семьи                   638
строительство собственной недвижимости    635
недвижимость                              633
операции со своей недвижимостью           627
строительство жилой недвижимости          624
покупка недвижимости                      621
покупка своего жилья                      620
строительство недвижимости                619
ремонт жилью                              607
покупка жилой недвижимости                606
на покупку своего автомобиля              505
заняться высшим образованием      

In [27]:
#проведем лемматизацию целей кредита 
all_lemmas = []
for row in bank_data['purpose'].unique().tolist():
    try:
        lemmas = m.lemmatize(row)
        all_lemmas.extend(lemmas)
    except:
        print('некорретные данные:', row)
print(Counter(all_lemmas))

Counter({' ': 59, '\n': 38, 'покупка': 10, 'недвижимость': 10, 'автомобиль': 9, 'образование': 9, 'жилье': 7, 'с': 5, 'операция': 4, 'на': 4, 'свой': 4, 'свадьба': 3, 'строительство': 3, 'получение': 3, 'высокий': 3, 'дополнительный': 2, 'для': 2, 'коммерческий': 2, 'жилой': 2, 'заниматься': 2, 'сделка': 2, 'приобретение': 1, 'сыграть': 1, 'проведение': 1, 'семья': 1, 'собственный': 1, 'подержать': 1, 'со': 1, 'подержанный': 1, 'профильный': 1, 'сдача': 1, 'ремонт': 1})


In [28]:
#из получившихся лем можно выделить несколько категорий целей кредита, создадим список этих категорий, они понадобятся нам дальше. 
categories = ['коммерческий','сдача','недвижимость', 'автомобиль', 'образование', 'свадьба',  'жилье']

### Вывод

Лемматизацию провели с помощью библиотеки pymystem3. Из каждой указанной цели кредита выделили основное слово, затем с помощью контейнера Counter посчитали количество всех лем, чтобы выделить наиболее часто встречеющиеся - это позволило выделить категории целей, на которые люди берут кредит. 

### Категоризация данных

In [29]:
#ранее мы уже создали категорию по возрасту: age_category
#теперь создадим категорию цели кредита

def purpose_category(text):
    lemma = m.lemmatize(text)
    for word in categories:
        if word in lemma:
            lemma = word
    return lemma

In [30]:
bank_data['purpose_category'] = bank_data['purpose'].apply(purpose_category)
bank_data.head(5)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_category,purpose_category
0,1,-8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья,35-44,жилье
1,1,-4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля,35-44,автомобиль
2,0,-5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья,25-34,жилье
3,3,-4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование,25-34,образование
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу,45-54,свадьба


In [31]:
#проверяем результат
bank_data['purpose_category'].value_counts()

недвижимость    5040
автомобиль      4306
образование     4013
жилье           3809
свадьба         2324
коммерческий    1311
сдача            651
Name: purpose_category, dtype: int64

In [32]:
#объединим категории 'недвижимость' и 'жилье' в категорию жилая недвижимость, 
#а категории 'коммерческий' и 'сдача' в категорию коммерческая недвижимость

bank_data.loc[(bank_data['purpose_category'] == 'недвижимость')|
              (bank_data['purpose_category'] == 'жилье'), 'purpose_category'] = 'жилая недвижимость'

bank_data.loc[(bank_data['purpose_category'] == 'коммерческий')|
              (bank_data['purpose_category'] == 'сдача'), 'purpose_category'] = 'коммерческая недвижимость'


In [33]:
#проверяем результат:
bank_data['purpose_category'].value_counts()

жилая недвижимость           8849
автомобиль                   4306
образование                  4013
свадьба                      2324
коммерческая недвижимость    1962
Name: purpose_category, dtype: int64

In [34]:
#теперь добавим категорию дохода
#для этого напишем функцию, которая присвоит каждому клиенту его категорию дохода:
def income_category(total_income):
    if 0 <= total_income < 100000:
        return 'до 100K'
    elif 100000 <= total_income < 150000:
        return '100 - 150K'
    elif 150000 <= total_income < 200000:
        return '150 - 200K'
    elif 200000 <= total_income < 250000:
        return '200 - 250K'
    elif total_income >= 250000:
        return '250K+'

In [35]:
bank_data['income_category'] = bank_data['total_income'].apply(income_category)

In [36]:
#проверяем
bank_data['income_category'].value_counts()

100 - 150K    7160
150 - 200K    4764
до 100K       4463
250K+         2813
200 - 250K    2254
Name: income_category, dtype: int64

### Вывод

1) Категоризация по возрасту была проведена следующим образом: нулевые значения были заменены на медианные по типу занятости, затем была написанная функция, присваивающая категорию возраста каждому клиенту.
Больше всего клиентов имеют возраст от 35 до 44 лет

2)Категоризация по целям кредита была проведена так: путем лемматизации мы выделили слова, описывающие цели кредитов, создали список категорий и с помощью функции присвоили категорию кредита каждому клиенту. Категории "жилая недвижимость" и "коммерческая недвижимость" мы специально разнесли в разные категории, так как клиенты, обратившиеся за кредитом на коммерческую недвижимость или на квартиру для сдачи, не нуждаются в жилье, а планируют с помощью этой недвижимости получать доход
Больше всего клиентов обратились за кредитом на жилую недвижимость

3)Категоризация по уровню дохода была проведена путем написания функции, присваивающей категорию дохода.  
Больше всего клиентов имеют доход в диапазоне 100-150К рублей. 


### Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [37]:
bank_data.pivot_table(index = ['children'], columns = 'debt', values = 'purpose', aggfunc = 'count')

debt,0,1
children,,
-1,46.0,1.0
0,13028.0,1063.0
1,4364.0,444.0
2,1858.0,194.0
3,303.0,27.0
4,37.0,4.0
5,9.0,NaN
20,68.0,8.0


In [38]:
#значения -1 и 20 в колонке children выглядят как ошибка ручного ввода, поэтому предположим, что  на самом деле должно быть "1" и "2".
#заменим это значение на 1 и пересчитаем данные: 

bank_data.loc[(bank_data['children'] == -1), 'children'] = 1
bank_data.loc[(bank_data['children'] == 20), 'children'] = 2

pivot = bank_data.pivot_table(index = ['children'], columns = 'debt', values = 'purpose', aggfunc = 'count')

#посчитаем долю невозвратов для каждой группы:
pivot['share'] = (pivot[1]/(pivot[0] + pivot[1])).round(3)
pivot.sort_values('share')

debt,0,1,share
children,,,
0,13028.0,1063.0,0.075
3,303.0,27.0,0.082
1,4410.0,445.0,0.092
2,1926.0,202.0,0.095
4,37.0,4.0,0.098
5,9.0,NaN,NaN


### Вывод

Из полученных данных видно, что чем больше у клиентов детей, чем меньше просрочек они делают. 
Из этого делаем вывод, что зависимость есть, и чем больше детей у заемщика, чем более надежным его можно считать.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [39]:
bank_pivot = bank_data.pivot_table(index = ['family_status'], columns = 'debt', 
                                   values = 'family_status_id', aggfunc = 'count').sort_values(1)

bank_pivot

debt,0,1
family_status,,
вдовец / вдова,896,63
в разводе,1110,85
Не женат / не замужем,2536,274
гражданский брак,3763,388
женат / замужем,11408,931


In [40]:
#так как количество взятых кредитов для каждой категории различается, 
#посчитаем какую долю составляют невозвраты от всех кредитов для каждой категории
bank_pivot['share'] = (bank_pivot[1]/(bank_pivot[0] + bank_pivot[1])).round(3)
bank_pivot.sort_values('share')

debt,0,1,share
family_status,,,
вдовец / вдова,896,63,0.066
в разводе,1110,85,0.071
женат / замужем,11408,931,0.075
гражданский брак,3763,388,0.093
Не женат / не замужем,2536,274,0.098


### Вывод

Из полученных данных видно, что люди, которые были или находятся в браке, имеют меньше просрочек, чем люди, в браке не состоящие. Вероятно, семейная жизнь учит отвественнее относиться к деньгам. 
Делаем вывод, что зависимость есть. 

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [41]:
bank_pivot1 = bank_data.pivot_table(index = ['income_category'], columns = 'debt', 
                                   values = 'purpose', aggfunc = 'count')

#посчитаем какую долю составляют невозвраты от всех кредитов для каждой категории
bank_pivot1['share'] = (bank_pivot1[1]/(bank_pivot1[0] + bank_pivot1[1])).round(3)
bank_pivot1.sort_values('share')

debt,0,1,share
income_category,,,
250K+,2619,194,0.069
200 - 250K,2090,164,0.073
до 100K,4109,354,0.079
150 - 200K,4359,405,0.085
100 - 150K,6536,624,0.087


### Вывод

Из полученной таблицы видим, что больше всего задолженностей у людей с доходом 100-200К рублей в месяц. Люди с более низким и более высоким доходом склонны более ответственно относиться к платежам, скорее всего потому, что людям с доходом до 100К рублей в месяц необходимо тщательно планировать свои расходы, а людям с доходом более 200К проще вносить платежи в срок + вероятно, они так же больше склонны планировать свои траты.

- Как разные цели кредита влияют на его возврат в срок?

In [42]:
bank_pivot2 = bank_data.pivot_table(index = ['purpose_category'], columns = 'debt', 
                                   values = 'purpose', aggfunc = 'count')

#посчитаем какую долю составляют невозвраты от всех кредитов для каждой категории
bank_pivot2['share'] = (bank_pivot2[1]/(bank_pivot2[0] + bank_pivot2[1])).round(3)
bank_pivot2.sort_values('share')

debt,0,1,share
purpose_category,,,
жилая недвижимость,8218,631,0.071
коммерческая недвижимость,1811,151,0.077
свадьба,2138,186,0.080
образование,3643,370,0.092
автомобиль,3903,403,0.094


### Вывод

Из получившейся таблицы видно, что сама низкая доля задолженностей у тех, кто брал кредит на жилье. Вероятно, большой платеж, долгий период выплат за жилье и важность этого самого жилья заставялют плательщиков ответственнее относиться к планированию собственных трат, чтобы внести платеж вовремя. То же касается и кредита на коммерческое жилье.
Интересно, что самое большое количество просрочек у тех, кто брал кредит на автомобиль, а не на свадьбу. 

In [43]:
bank_data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_category,purpose_category,income_category
0,1,-8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья,35-44,жилая недвижимость,250K+
1,1,-4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля,35-44,автомобиль,100 - 150K
2,0,-5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья,25-34,жилая недвижимость,100 - 150K
3,3,-4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование,25-34,образование,250K+
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу,45-54,свадьба,150 - 200K


### Шаг 4. Общий вывод

Исходные данные содержали ошибки: пропущенные значения, дубликаты, ошибки заполнения, некорректные данные (например, отрицательные значения в столбце days_employed, children, нулевые значения в возрасте заемщика)

Мы произвели предобработку данных, сделав их удобными для анализа: исправили большинство ошибок, заполнили пропущенные значения, выделили категории возраста, кредитных целей и дохода. 

При исследовании надежности заемщиков мы увидели, что самыми надежными являются те, кто берет кредит на недвижимость, имеет доход от 250К рублей в месяц, был или находится в браке и имеет детей. 
Самые ненадежные заемщики это люди, берущие кредит на автомобиль, имеющие доход 100-200К рублей в месяц, не состоит в браке и не имеющие детей. 